In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("test.csv")

In [ ]:
df.head()

,review_description,rating
0,انتم في قمة الروعه تستاهلون مليون نجمه,1
1,جربت غيره و فضلته لعدة أسباب تطبيق مرتب وسهل و...,1
2,ههههه😂 يوم اخوياي فصلو علي وصل 😂,1
3,أسوأ خدمة على الإطلاق ، انتظرت في الشارع لأكثر...,-1
4,بس وصلو بسرعة لوسمحتون,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6408 entries, 0 to 6407
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   review_description  6408 non-null   object
 1   rating              6408 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 100.3+ KB


In [ ]:
# Check nulls and datatypes
nulls_info = pd.DataFrame({
    "Null_Count": df.isnull().sum(),
    "DataType": df.dtypes
})

print(nulls_info)


                    Null_Count DataType
review_description           0   object
rating                       0    int64


In [ ]:
duplicates_count = df.duplicated().sum()
print("Total duplicate rows:", duplicates_count)

Total duplicate rows: 39


In [ ]:
df = df.drop_duplicates()

In [ ]:
import re

def extract_non_arabic(text):
    """
    Extract all characters that are NOT Arabic letters.
    """
    if not isinstance(text, str):
        return ""
    return ''.join(re.findall(r'[^\u0600-\u06FF]', text))

all_text = ''.join(df['review_description'].astype(str).tolist())
non_arabic_chars = set(extract_non_arabic(all_text))

print(non_arabic_chars)
print(f"\nTotal unique non-Arabic characters found: {len(non_arabic_chars)}")


{'★', 'J', 'e', 'ﻴ', '😦', '👠', '🍪', 'ﻵ', '🍔', '☹', '0', '🚧', '£', 'Q', 'D', 'm', '♥', 'y', '😒', '👈', '👉', '🦞', '🤲', 'U', '😨', '💪', 'P', '🤍', '🥪', '》', '🤔', '⚘', '✯', '😷', '👄', '🍩', '🥺', '😖', '🐖', '🏻', '🐥', '🙇', '🌭', '5', 'f', '🇪', '🇧', '🍸', 'R', '😻', '|', '🐟', '💢', '🇭', '🥞', '🍷', '🌮', '🎀', '🧁', '🟧', '🇸', '🍻', '🍝', '🖤', '8', '℅', '😡', 'c', '🥶', '😶', '😙', '📵', '😚', '🤗', '😀', 'W', '■', '🙌', '🎁', '🍯', '🍒', '🖓', '💿', '🐛', '=', '😇', '😲', '💗', 'B', '🍳', '❎', '😕', '@', '🍖', '🎉', '🍦', '🍧', ' ', '🌼', '🏽', '👹', '+', '🌯', '👕', '🌱', '🍨', '●', '$', 'G', '🥗', '🍛', '🌹', 's', '👗', '😂', '4', '🔥', '☑', '♡', '🌞', 'u', '😗', 'Y', '🙉', '🙊', '하', '😆', 'ﻻ', '😘', '💀', '💔', '🍴', 'M', 'p', '😳', '□', '🤧', '¤', '👸', '7', '🖕', '🍹', '💖', '…', '🙆', '🥩', '💜', 'a', '🤬', '😢', 'E', '🍙', 't', 'F', '🍲', '😤', '☆', '🙄', '&', '🥲', 'K', '3', 'o', '6', '🍕', 'ﷲ', '✊', '🤚', '🙈', '🍗', 'h', '-', 'b', '🐎', '😍', '👌', '🍜', 'Z', '🤦', 'V', '🐷', '🇺', '😔', '🇴', '🍰', '❔', '💌', '😑', '😁', '💲', '🥙', 'ﺟ', '️', '💃', 'A', '«', '🤑', '😬', '_', '◍',

In [ ]:
# !pip install camel-tools
# !camel_data -i all


In [ ]:

from camel_tools.utils import normalize

# Example: normalize Arabic text (fix alif, yaa, etc.)
df['review_description'] = df['review_description'].apply(lambda x: normalize.normalize_alef_maksura_ar(x))
df['review_description'] = df['review_description'].apply(lambda x: normalize.normalize_alef_ar(x))
df['review_description'] = df['review_description'].apply(lambda x: normalize.normalize_teh_marbuta_ar(x))

# Each function handles a specific variant:
# normalize_alef_maksura_ar → convert ى → ي
# normalize_alef_ar         → convert أ إ آ → ا
# normalize_teh_marbuta_ar  → convert ة → ه (اختياري)


In [ ]:
from camel_tools.utils.dediac import dediac_ar

# Remove diacritics automatically
df['review_description'] = df['review_description'].apply(lambda x: dediac_ar(x))


In [ ]:
# If not installed:
# pip install arabic-reshaper

# Simple digit map (auto usable)
digit_map = str.maketrans('0123456789', '٠١٢٣٤٥٦٧٨٩')

def convert_digits_to_arabic(text):
    """
    Convert ASCII (English) digits in a string to Arabic-Indic digits.
    Example: '123' -> '١٢٣'
    """
    if not isinstance(text, str):
        return text
    return text.translate(digit_map)


df['review_description'] = df['review_description'].apply(convert_digits_to_arabic)


In [ ]:
# Install the emoji library if needed:
# pip install emoji

import emoji

def emojis_to_arabic(text: str) -> str:
    """
    Replace emojis with Arabic descriptions using emoji.demojize with Arabic language setting.
    """
    if not isinstance(text, str):
        return text
    # Convert emojis to text codes in Arabic
    return emoji.demojize(text, language='ar')

df['review_description'] = df['review_description'].apply(emojis_to_arabic)


In [ ]:
df.head()

,review_description,rating
0,انتم في قمه الروعه تستاهلون مليون نجمه,1
1,جربت غيره و فضلته لعده اسباب تطبيق مرتب وسهل و...,1
2,ههههه:وجه_بدموع_فرح: يوم اخوياي فصلو علي وصل :...,1
3,اسوا خدمه علي الاطلاق ، انتظرت في الشارع لاكثر...,-1
4,بس وصلو بسرعه لوسمحتون,1


In [ ]:
import re

# Map for currency symbols
currency_map = {
    '€': ' يورو ',
    '¥': ' ين ',
    '₩': ' وون ',
    '﷼': ' ريال ',
    '$': ' دولار ',
    '£': ' جنيه ',
    '¢': ' سنت '
}

def replace_currencies(text):
    if not isinstance(text, str):
        return text
    for sym, name in currency_map.items():
        text = text.replace(sym, name)
    return text

# Step 1: Convert tokens like :قلب_أخضر: → " قلب أخضر "
def replace_emoji_tokens(text):
    if not isinstance(text, str):
        return text
    return re.sub(r':([^:]+):', lambda m: ' ' + m.group(1).replace('_', ' ') + ' ', text)

# Step 2: Remove thin spaces, newlines, punctuation (but KEEP normal spaces)
def remove_unwanted_chars(text):
    if not isinstance(text, str):
        return text
    return re.sub(r'[\u200a\n.,!?؟]', '', text)

# Step 3: Keep only Arabic letters, Arabic digits, and spaces
def keep_only_arabic(text):
    if not isinstance(text, str):
        return text
    return re.sub(r'[^\u0600-\u06FF٠-٩ ]', '', text)

# Combine all steps
def clean_arabic_text(text):
    text = replace_currencies(text)
    text = replace_emoji_tokens(text)   # expand emojis first with spaces
    text = remove_unwanted_chars(text)
    text = keep_only_arabic(text)
    text = re.sub(r'\s+', ' ', text).strip()  # normalize spaces
    return text

# Apply on the dataframe column
df['review_description'] = df['review_description'].apply(clean_arabic_text)


In [ ]:
import re

def extract_non_arabic(text):
    """
    Extract all characters that are NOT Arabic letters.
    """
    if not isinstance(text, str):
        return ""
    return ''.join(re.findall(r'[^\u0600-\u06FF]', text))

all_text = ''.join(df['review_description'].astype(str).tolist())
non_arabic_chars = set(extract_non_arabic(all_text))

print(non_arabic_chars)
print(f"\nTotal unique non-Arabic characters found: {len(non_arabic_chars)}")


{' '}

Total unique non-Arabic characters found: 1


In [ ]:
df.head()

,review_description,rating
0,انتم في قمه الروعه تستاهلون مليون نجمه,1
1,جربت غيره و فضلته لعده اسباب تطبيق مرتب وسهل و...,1
2,ههههه وجه بدموع فرح يوم اخوياي فصلو علي وصل وج...,1
3,اسوا خدمه علي الاطلاق ، انتظرت في الشارع لاكثر...,-1
4,بس وصلو بسرعه لوسمحتون,1


In [ ]:
import requests
from camel_tools.tokenizers.word import simple_word_tokenize
import pandas as pd

url = 'https://raw.githubusercontent.com/mohataher/arabic-stop-words/master/list.txt'
response = requests.get(url)
arabic_stopwords = set(response.text.splitlines())

def tokenize_remove_stopwords(text):
    tokens = simple_word_tokenize(text)
    tokens = [w for w in tokens if w not in arabic_stopwords]
    return tokens

df['tokens'] = df['review_description'].apply(tokenize_remove_stopwords)


df['cleaned_review'] = df['tokens'].apply(lambda x: ' '.join(x))

print(df[['review_description', 'cleaned_review']])


                                     review_description  \
0                انتم في قمه الروعه تستاهلون مليون نجمه   
1     جربت غيره و فضلته لعده اسباب تطبيق مرتب وسهل و...   
2     ههههه وجه بدموع فرح يوم اخوياي فصلو علي وصل وج...   
3     اسوا خدمه علي الاطلاق ، انتظرت في الشارع لاكثر...   
4                                بس وصلو بسرعه لوسمحتون   
...                                                 ...   
6403                                               عحيب   
6404  التطبيق رائع والشركه مهما تكلمت عنها لا اوفيها...   
6405                              برنامج ضعيف وغير واضح   
6406         ليش مش قابل ينزل عندي و بضل يحمل و ما بنزل   
6407                                      هو ممتاز كتير   

                                         cleaned_review  
0                         انتم قمه الروعه تستاهلون نجمه  
1     جربت غيره فضلته لعده اسباب تطبيق مرتب وسهل دقه...  
2     ههههه وجه بدموع فرح اخوياي فصلو وصل وجه بدموع فرح  
3       اسوا خدمه الاطلاق انتظرت الشارع لاكثر ساعه ونصف  
4

In [ ]:
df.head()

,review_description,rating,tokens,cleaned_review
0,انتم في قمه الروعه تستاهلون مليون نجمه,1,"[انتم, قمه, الروعه, تستاهلون, نجمه]",انتم قمه الروعه تستاهلون نجمه
1,جربت غيره و فضلته لعده اسباب تطبيق مرتب وسهل و...,1,"[جربت, غيره, فضلته, لعده, اسباب, تطبيق, مرتب, ...",جربت غيره فضلته لعده اسباب تطبيق مرتب وسهل دقه...
2,ههههه وجه بدموع فرح يوم اخوياي فصلو علي وصل وج...,1,"[ههههه, وجه, بدموع, فرح, اخوياي, فصلو, وصل, وج...",ههههه وجه بدموع فرح اخوياي فصلو وصل وجه بدموع فرح
3,اسوا خدمه علي الاطلاق ، انتظرت في الشارع لاكثر...,-1,"[اسوا, خدمه, الاطلاق, انتظرت, الشارع, لاكثر, س...",اسوا خدمه الاطلاق انتظرت الشارع لاكثر ساعه ونصف
4,بس وصلو بسرعه لوسمحتون,1,"[بس, وصلو, بسرعه, لوسمحتون]",بس وصلو بسرعه لوسمحتون


In [ ]:
# Show 5 random rows
df.sample(5)

,review_description,rating,tokens,cleaned_review
1880,توصيل سريع,1,"[توصيل, سريع]",توصيل سريع
3925,قبول قلب أرجواني,1,"[قبول, قلب, أرجواني]",قبول قلب أرجواني
4804,تطبيق رائع لكن مايخدم التوصيل في السعوديه,-1,"[تطبيق, رائع, مايخدم, التوصيل, السعوديه]",تطبيق رائع مايخدم التوصيل السعوديه
3933,سهل الاستخدام وسريع في التوصيل,1,"[سهل, الاستخدام, وسريع, التوصيل]",سهل الاستخدام وسريع التوصيل
1747,تطبيق جميللل,1,"[تطبيق, جميللل]",تطبيق جميللل


In [ ]:
!camel_data -i all


The following packages will be installed: 'morphology-db-lev-01', 'morphology-db-msa-r13', 'morphology-db-msa-s31', 'dialectid-model26', 'sentiment-analysis-mbert', 'sentiment-analysis-arabert', 'ner-arabert', 'disambig-ranking-cache-calima-glf-01', 'disambig-bert-unfactored-egy', 'disambig-ranking-cache-calima-msa-r13', 'disambig-mle-calima-msa-r13', 'disambig-bert-unfactored-glf', 'disambig-bert-unfactored-msa', 'disambig-ranking-cache-calima-egy-r13', 'morphology-db-egy-r13', 'dialectid-model6'
Extracting package 'morphology-db-lev-01': 100% 10.6M/10.6M [00:00<00:00, 155MB/s]
Extracting package 'morphology-db-msa-r13': 100% 40.5M/40.5M [00:00<00:00, 430MB/s]
Extracting package 'morphology-db-msa-s31': 100% 44.8M/44.8M [00:00<00:00, 251MB/s]
Extracting package 'dialectid-model26': 100% 371M/371M [00:01<00:00, 284MB/s]
Extracting package 'sentiment-analysis-mbert': 100% 712M/712M [00:02<00:00, 280MB/s]
Extracting package 'sentiment-analysis-arabert': 100% 542M/542M [00:01<00:00, 299MB

In [ ]:
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer
from camel_tools.tokenizers.word import simple_word_tokenize

# Load the standard Arabic morphology database
db = MorphologyDB.builtin_db()
analyzer = Analyzer(db)

def lemmatize_arabic(text):
    tokens = simple_word_tokenize(text)
    lemmas = []
    for token in tokens:
        analyses = analyzer.analyze(token)
        if analyses:
            # تحقق إذا مفتاح 'lemma' موجود
            lemma = analyses[0].get('lemma', token)
            lemmas.append(lemma)
        else:
            lemmas.append(token)
    return ' '.join(lemmas)

# Apply lemmatization on the column
df['lemmatized'] = df['cleaned_review'].apply(lemmatize_arabic)

print(df[['cleaned_review', 'lemmatized']].head())


# Apply lemmatization on the column
df['lemmatized'] = df['cleaned_review'].apply(lemmatize_arabic)

print(df[['cleaned_review', 'lemmatized']].head())


                                      cleaned_review  \
0                      انتم قمه الروعه تستاهلون نجمه   
1  جربت غيره فضلته لعده اسباب تطبيق مرتب وسهل دقه...   
2  ههههه وجه بدموع فرح اخوياي فصلو وصل وجه بدموع فرح   
3    اسوا خدمه الاطلاق انتظرت الشارع لاكثر ساعه ونصف   
4                             بس وصلو بسرعه لوسمحتون   

                                          lemmatized  
0                      انتم قمه الروعه تستاهلون نجمه  
1  جربت غيره فضلته لعده اسباب تطبيق مرتب وسهل دقه...  
2  ههههه وجه بدموع فرح اخوياي فصلو وصل وجه بدموع فرح  
3    اسوا خدمه الاطلاق انتظرت الشارع لاكثر ساعه ونصف  
4                             بس وصلو بسرعه لوسمحتون  
                                      cleaned_review  \
0                      انتم قمه الروعه تستاهلون نجمه   
1  جربت غيره فضلته لعده اسباب تطبيق مرتب وسهل دقه...   
2  ههههه وجه بدموع فرح اخوياي فصلو وصل وجه بدموع فرح   
3    اسوا خدمه الاطلاق انتظرت الشارع لاكثر ساعه ونصف   
4                             بس وصلو بسرعه لوسمحتون 

In [ ]:
df.sample(5)

,review_description,rating,tokens,cleaned_review,lemmatized
5981,خوش برنامج وفر علينا وقت وجهد قبول,1,"[خوش, برنامج, وفر, علينا, وقت, وجهد, قبول]",خوش برنامج وفر علينا وقت وجهد قبول,خوش برنامج وفر علينا وقت وجهد قبول
2148,اسوء خدمهبتتقدم عن طريقهم اول مرهمفروض معايا خ...,-1,"[اسوء, خدمهبتتقدم, طريقهم, مرهمفروض, معايا, خص...",اسوء خدمهبتتقدم طريقهم مرهمفروض معايا خصم اتبع...,اسوء خدمهبتتقدم طريقهم مرهمفروض معايا خصم اتبع...
3092,اسوا خدمه حافلات للمدارس,-1,"[اسوا, خدمه, حافلات, للمدارس]",اسوا خدمه حافلات للمدارس,اسوا خدمه حافلات للمدارس
3427,رررروووعععهههه,1,[رررروووعععهههه],رررروووعععهههه,رررروووعععهههه
2633,ممتاز خرافي,1,"[ممتاز, خرافي]",ممتاز خرافي,ممتاز خرافي


In [ ]:
df = df[["rating", "lemmatized"]]

In [ ]:
df.sample(5)

,rating,lemmatized
2457,1,جميل ومفيد ويختصر
5612,-1,برنامج زباله
4567,1,يجنن٣
532,1,احب فعلا تطبيق ممتاااااز
5993,1,خدمات مميزه يشكرون


In [ ]:
import torch
import numpy as np
import joblib
from transformers import AutoTokenizer, AutoModel


tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")
model = AutoModel.from_pretrained("aubmindlab/bert-base-arabertv2")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(64000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
def get_embeddings_batch(texts, batch_size=32):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        #
        inputs = tokenizer(batch_texts, return_tensors="pt",
                           truncation=True, padding=True, max_length=64)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # CLS token
            all_embeddings.append(batch_embeddings)

    return np.vstack(all_embeddings)


In [ ]:
test_texts = df["lemmatized"].tolist()
X_test = get_embeddings_batch(test_texts, batch_size=64)



In [ ]:
embeddings = get_embeddings_batch(test_texts, batch_size=64)

In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout


X = np.array(embeddings)
y_map = {-1: 0, 0: 1, 1: 2}
y = df["rating"].map(y_map).values

if len(X.shape) == 2:
    X = np.expand_dims(X, axis=1)

In [ ]:
from tensorflow.keras.models import load_model
model_lstm = load_model("lstm_classification_model.h5")

model_rnn = load_model("rnn_classification_model.h5")


In [ ]:
# RNN
loss_rnn, acc_rnn = model_rnn.evaluate(X, y, verbose=0)
print(f"RNN Test Accuracy: {acc_rnn*100:.2f}%")


RNN Test Accuracy: 81.00%


In [ ]:
# LSTM
loss_lstm, acc_lstm = model_lstm.evaluate(X, y, verbose=0)
print(f"LSTM Test Accuracy: {acc_lstm*100:.2f}%")



LSTM Test Accuracy: 79.90%
